<center><h1>VC07: Biclustering</h1></center>

En esta práctica estudiaremos el funcionamiento y la utilización de diferentes algoritmos de biclustering ya implementados en diferentes librerías. Por un lado, conoceremos la librería <b>biclustlib</b>, que es necesario instalar a partir del siguiente repositorio:

https://github.com/padilha/biclustlib



In [ ]:
import numpy as np
import pandas as pd
import itertools as it
import matplotlib.pyplot as plt


En la librería <b>biclustlib</b> encontramos una implementación del algoritmo de Cheng y Church. Además usaremos el conjunto de datos que viene con la librería relativo a la expresión genética de la levadura, de uso habitual en biotecnología.


In [ ]:
from biclustlib.algorithms import ChengChurchAlgorithm
from biclustlib.datasets import load_yeast_tavazoie

# Cargamos los datos de levadura ('yeast' en inglés)
data = load_yeast_tavazoie().values

# los valores perdidos se imputan de manera aleatoria 
# de acuerdo a la recomendación de Cheng y Church
missing = np.where(data < 0.0)
data[missing] = np.random.randint(low=0, high=800, size=len(missing[0]))

# Esta parametrización específica es la que originalmente 
# describían Cheng y Church en su artículo
modelo = ChengChurchAlgorithm(num_biclusters=100,
                              msr_threshold=300.0, 
                              multiple_node_deletion_threshold=1.2)
biclustering = modelo.run(data)


Una vez aprendido el modelo, podemos observar qué biclústeres se obtuvieron como resultado final.


In [ ]:
ind_bicluster = 0 # índice del biclúster a examinar

print(biclustering.biclusters[ind_bicluster])
print("El área que cubre este bicluster es:",biclustering.biclusters[ind_bicluster].area, 
     "sobre un total de",np.product(data.shape))

plt.figure()
plt.matshow(data[np.ix_(biclustering.biclusters[ind_bicluster].rows,
                        biclustering.biclusters[ind_bicluster].cols)], 
            cmap=plt.cm.Blues)
plt.title("Bicluster")
plt.show()



También podemos generar nuestros propios datos artificiales para estudiar el funcionamiento de los algoritmos usando la función <b>make_biclusters</b> de sklearn:


In [ ]:
from sklearn.datasets import make_biclusters
size = 300

# Usamos la función make_biclusters para generar una matriz con biclusters aislados
# (no todas las filas ni columnas pertenecen a algún clúster)
n_clusters = 5
orig_data, rows, columns = make_biclusters(shape=(size, size), n_clusters=n_clusters, 
                                           noise=10, shuffle=False, random_state=0)

# Inducimos un desorden aleatorio de filas y de columnas
sh_rows = np.random.choice(size, size=size, replace=False)
sh_cols = np.random.choice(size, size=size, replace=False)
transf_data = orig_data[sh_rows,:]
transf_data = transf_data[:,sh_cols]


# En esta ocasión estamos usando información privilegiada: sabemos cuántos clústeres 
# se generaron
modelo = ChengChurchAlgorithm(num_biclusters=n_clusters, 
                              msr_threshold=200.0, 
                              multiple_node_deletion_threshold=1.5)
biclustering = modelo.run(transf_data)


Puedes variar los párametros del modelo, tanto <b>msr_threshold</b> como <b>multiple_node_deletion_threshold</b> para observar el comportamiento y el tipo de coagrupamientos que se obtienen.


In [ ]:
ind_bicluster = 0 # índice del biclúster a examinar

print(biclustering.biclusters[ind_bicluster])
print("El área que cubre este bicluster es:",biclustering.biclusters[ind_bicluster].area, 
     "sobre un total de",np.product(transf_data.shape))

plt.figure()
plt.matshow(transf_data[np.ix_(biclustering.biclusters[ind_bicluster].rows,
                        biclustering.biclusters[ind_bicluster].cols)], 
            cmap=plt.cm.Blues)
plt.title("Dataset original")
plt.show()


<hr>

Por su parte, la librería ScikitLearn implementa el algoritmo de coagrupamiento espectral. De la parte teórica de la asignatura sabemos que este tipo de algoritmo es especialmente útil cuando los clústeres tienen forma de tablero de ajedrez. 

Podemos generar datos artificiales de este estilo usando la función <b>make_checkerboard</b> de sklearn:


In [ ]:
from sklearn.datasets import make_checkerboard, make_biclusters
from sklearn.cluster.bicluster import SpectralBiclustering, SpectralCoclustering
from sklearn.metrics import consensus_score

np.random.seed(17)

size = 300

# Usamos la función make_checkerboard para generar una matriz con la forma de tablero
# de ajedrez que, como sabemos, suele darse en escenarios de coagrupamiento
n_clusters = (4, 3)
orig_data, rows, columns = make_checkerboard(shape=(size, size), n_clusters=n_clusters, 
                                             noise=100, shuffle=False, random_state=0)
# Usaríamos la función make_biclusters para generar una matriz con biclusters aislados
# (no todas las filas ni columnas pertenecen a algún clúster)
# n_clusters = 5
# orig_data, rows, columns = make_biclusters(shape=(size, size), n_clusters=n_clusters, 
#                                            noise=10, shuffle=False, random_state=0)

# Buscamos un desorden aleatorio de filas y de columnas
sh_rows = np.random.choice(size, size=size, replace=False)
sh_cols = np.random.choice(size, size=size, replace=False)
transf_data = orig_data[sh_rows,:]
transf_data = transf_data[:,sh_cols]

# Realizamos el biclustering sobre los datos desordenados
# Asumimos que conocemos el número de clústeres (por filas y columnas)
modelo = SpectralBiclustering(n_clusters=n_clusters, method='log',
                              random_state=0)
# También podríamos usar el algoritmo de SpectralCoclustering
# modelo = SpectralCoclustering(n_clusters=n_clusters, random_state=0)

modelo.fit(transf_data)

# Aprovechamos que estamos en un entorno simulado y conocemos los 
# biclústeres originales para comprobar el parecido de los biclústeres 
# obtenidos con respecto a los originales
score = consensus_score(modelo.biclusters_,
                        (rows[:, sh_rows], columns[:, sh_cols]))

print("El parecido según la Medida de Consenso es: ", format(score))

# Reconstruimos la matriz original según el agrupamiento por filas 
# y columnas devuelto por el modelo  
fit_data = transf_data[np.argsort(modelo.row_labels_)]
fit_data = fit_data[:, np.argsort(modelo.column_labels_)]

struct_data = np.outer(np.sort(modelo.row_labels_) + 1,
                       np.sort(modelo.column_labels_) + 1)


También podemos dibujar la matriz de datos en las diferentes etapas para ver cómo se produce la reconstrucción de los datos:
    

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1,4,figsize=(20,20))

ax1.matshow(orig_data, cmap=plt.cm.Blues)
ax1.set_title("Dataset original")

ax2.matshow(transf_data, cmap=plt.cm.Blues)
ax2.set_title("Dataset desordenado")

ax3.matshow(fit_data, cmap=plt.cm.Blues)
ax3.set_title("Reordenado según el resultado del biclustering")

ax4.matshow(struct_data, cmap=plt.cm.Blues)
ax4.set_title("Estructura inferida del biclustering obtenido")

plt.show()


Puedes probar a usar diferente parámetros (introducir más o menos ruido, por ejemplo) para observar el comportamiento del algoritmo.


<hr/>

Por último en este tutorial, veremos otra aplicación práctica relacionada con el análisis de biclústeres. En este caso se trata de agrupar un conjunto de noticias y encontrar, a su vez, sobre qué hablan esos grupos de noticias:


In [ ]:
from sklearn.datasets.twenty_newsgroups import fetch_20newsgroups

import re
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.cluster import MiniBatchKMeans

from sklearn.metrics.cluster import normalized_mutual_info_score

In [ ]:
# Accedemos a los datos
categories = ['alt.atheism', 'comp.graphics', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
              'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball',
              'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 
              'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast',
              'talk.politics.misc', 'talk.religion.misc']
newsgroups = fetch_20newsgroups(categories=categories)
y_true = newsgroups.target

# Tokenizer que, además, reemplaza todos los tokens numéricos por una misma etiqueta
def tokenizer(doc):
    patrones = re.compile(u'(?u)\\b\\w\\w+\\b')
    tokens = patrones.findall(doc)
    tokens = ["#NUMBER" if token[0] in "0123456789_" else token for token in tokens]
    return tokens

# Obtenemos una matriz a partir de los documentos: un documento por fila
vectorizer = TfidfVectorizer(stop_words='english', min_df=5, tokenizer=tokenizer)
X = vectorizer.fit_transform(newsgroups.data)

# Llevamos a cabo el co-agrupamiento
modelo_cocluster = SpectralCoclustering(n_clusters=len(categories), 
                                 svd_method='arpack', random_state=0)
modelo_cocluster.fit(X)
print("Información mutual (norm.) de SpectralCoclustering: ", 
      format(normalized_mutual_info_score(y_true, modelo_cocluster.row_labels_,
                                          average_method='arithmetic')))

# por comparación, aprendemos un k-means para observar si la consideración 
# de usar coagrupamiento en vez de un simple agrupamiento conduce a mejores resultados
modelo_kmeans = MiniBatchKMeans(n_clusters=len(categories), batch_size=20000,
                         random_state=0)
y_kmeans = modelo_kmeans.fit_predict(X)
print("Información mutual (norm.) de K-means: ", 
      format(normalized_mutual_info_score(y_true, y_kmeans,
                                          average_method='arithmetic')))


Finalmente, podemos observar los tipos de artículos en los diferentes co-agrupamientos. Necesitaremos una serie de funciones auxiliares:


In [ ]:
from sklearn.externals.six import iteritems
from collections import defaultdict
import operator

def bicluster_ncut(i, cocluster):
    rows, cols = cocluster.get_indices(i)
    if not (np.any(rows) and np.any(cols)):
        import sys
        return sys.float_info.max
    row_complement = np.nonzero(np.logical_not(cocluster.rows_[i]))[0]
    col_complement = np.nonzero(np.logical_not(cocluster.columns_[i]))[0]
    weight = X[rows][:, cols].sum()
    cut = (X[row_complement][:, cols].sum() +
           X[rows][:, col_complement].sum())
    return cut / weight


def most_common(d):
    return sorted(iteritems(d), key=operator.itemgetter(1), reverse=True)



Finalmente, mediante estas líneas mostraremos la lista de los mejores biclústeres con información sobre los temas principales que tratan los documentos de ese biclúster y las palabras más comunes:


In [ ]:
mostrar_ejemplos = 5 # Número de biclústeres a mostrar

palabras = vectorizer.get_feature_names()
documentos = list(newsgroups.target_names[i] for i in newsgroups.target)

bicluster_ncuts = list(bicluster_ncut(i, modelo_cocluster)
                       for i in range(len(newsgroups.target_names)))
mejores_biclusteres = np.argsort(bicluster_ncuts)[:mostrar_ejemplos]

print()
print("Los mejores biclústeres son:")
print()
for ind, cluster in enumerate(mejores_biclusteres):
    n_rows, n_cols = modelo_cocluster.get_shape(cluster)
    bic_documentos, bic_palabras = modelo_cocluster.get_indices(cluster)

    # categorias
    counter = defaultdict(int)
    for i in bic_documentos:
        counter[documentos[i]] += 1
    combinacion_categorias = ", ".join("{:.0f}% {}".format(float(c) / n_rows * 100, nombre)
                                       for nombre, c in most_common(counter)[:3])

    # palabras
    bic_documentos_out = np.where(modelo_cocluster.row_labels_ != cluster)[0]
    X_palabras = X[:, bic_palabras]
    score_palabras = np.array(X_palabras[bic_documentos, :].sum(axis=0) -
                              X_palabras[bic_documentos_out, :].sum(axis=0))
    score_palabras = score_palabras.ravel()
    lista_palabras = list(palabras[bic_palabras[i]]
                          for i in score_palabras.argsort()[:-11:-1])
    lista_palabras = ', '.join(lista_palabras)

    print("Biclúster", ind, "(", n_rows, "documentos,", n_cols, "palabras)")
    print(" + Categorias:", combinacion_categorias)
    print(" + Palabras  :", lista_palabras)
    print()